In [1]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
# from ROI_extraction import preprocess_image
import cv2
import os
from ROI_extraction import DataGenerator
# Set the path to dataset
dataset_path = '../images/3regimes'

ids = []
labels = {}
classes = {'excess':1,'normal':0,'insufficient':-1}
for class_name in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_name)
    if os.path.isdir(class_path):
        for filename in os.listdir(class_path) :
            if filename.endswith((".jpg", ".jpeg", ".png")):
                img_path = os.path.join(class_path, filename) 
                ids.append(img_path)
                labels[img_path]=classes[class_name]

# # Shuffle the list of tuples
# random.shuffle(ids)

# # Define the split ratio (e.g., 80% for training, 20% for validation)
# split_ratio = 0.8

# # Calculate the index for splitting
# split_index = int(len(ids) * split_ratio)

# # Split the shuffled IDs and labels into training and validation sets
# train_ids = ids[:split_index]
# val_ids = ids[split_index:]

In [2]:
def correct_gamma(image):
    # Convert image to float and normalize to range 0-1
    image_normalized = image.astype(float) / 255.0

    # Calculate mean R intensity
    meanRimg = np.mean(image_normalized[:, :, 2])  # Image is in BGR format
    
    # Calculate G value
    G = 0.74 * np.exp(-3.97 * meanRimg)
    
    # Apply transformation
    transformed_image = np.power(image_normalized, 1 / G)
    img_float32 = np.float32(transformed_image)
    return img_float32

def extract_ROI(original_image):
    # Convert to grayscale
    gray_image = cv2.cvtColor((original_image*255).astype(np.uint8), cv2.COLOR_BGR2GRAY)
    
    # # Apply histogram normalization
    # normalized_image = cv2.equalizeHist(gray_image)
    
    # Apply median filtering
    filtered_image = cv2.medianBlur(gray_image, 5)
    
    # Apply Otsu's thresholding
    _, thresholded_image = cv2.threshold(filtered_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Apply morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    closed_image = cv2.morphologyEx(thresholded_image, cv2.MORPH_CLOSE, kernel)
    opened_image = cv2.morphologyEx(closed_image, cv2.MORPH_OPEN, kernel)

    # Find contours in the processed image
    contours, _ = cv2.findContours(opened_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the contour with the largest area
    contour = max(contours, key=cv2.contourArea)
    
    # Calculate the moments of the contour
    M = cv2.moments(contour)
    
    # Calculate the center of the contour
    center_x = int(M["m10"] / M["m00"])
    center_y = int(M["m01"] / M["m00"])
    
    # Calculate the coordinates of the square ROI
    roi_size = 100
    roi_x = center_x - roi_size // 2
    roi_y = center_y - roi_size // 2
    
    return {'contours':contours,'roi_x':roi_x,'roi_y':roi_y,'roi_size':roi_size}

In [3]:
# Set the input image dimensions
img_width, img_height = 100, 100
n_channels = 3

params = {'dim': (img_height,img_width),
          'batch_size': 64,
          'n_classes': 3,
          'n_channels': n_channels,
          'shuffle': False}

# Set the number of classes
num_classes = 3

In [4]:
def data_generation(list_IDs_temp):
    X = []
    y = np.empty((len(list_IDs_temp)), dtype=int)
    for i, ID in enumerate(list_IDs_temp):
        image = cv2.imread(ID)
        img_gamma_correct = correct_gamma(image)
        ROI = extract_ROI(img_gamma_correct)
        ROI = image[ROI['roi_y']:ROI['roi_y']+ROI['roi_size'], ROI['roi_x']:ROI['roi_x']+ROI['roi_size']]
        # ROI = cv2.cvtColor(ROI, cv2.COLOR_BGR2GRAY)
        X.append(ROI)
        # Store class
        y[i] = labels[ID]
    X = np.reshape(X,(len(list_IDs_temp),img_width, img_height,params['n_channels']))
    X = X.astype("float32") / 255.0
    return X, y


In [5]:
X,y = data_generation(ids)

In [6]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
# Nombre de plis pour la validation croisée k-fold
k = 5

# Créer une instance de StratifiedKFold avec k plis
skf = StratifiedKFold(n_splits=k)



In [8]:
# Liste pour stocker les scores de validation
scores = {i:{'history':None,'history_fine_tuning':None} for i in range(k)}

In [9]:
scores

{0: {'history': None, 'history_fine_tuning': None},
 1: {'history': None, 'history_fine_tuning': None},
 2: {'history': None, 'history_fine_tuning': None},
 3: {'history': None, 'history_fine_tuning': None},
 4: {'history': None, 'history_fine_tuning': None}}

In [29]:
nb_classes = {-1:0,0:0,1:0}
y_train_test = keras.utils.to_categorical(y, num_classes=params['n_classes'])
for c in y_train_test:
    if(np.array_equal([0., 0., 1.],c)):
        nb_classes[0]+=1
    if(np.array_equal([0., 1., 0.],c)):
        nb_classes[-1]+=1
    if(np.array_equal([1., 0., 0.],c)):
        nb_classes[1]+=1
print(nb_classes)

{-1: 1670, 0: 1670, 1: 1670}


In [7]:
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")
    # Diviser les données d'entraînement et de validation pour ce pli
    X_train, X_val = X[train_index], X[test_index]
    y_train, y_val = y[train_index], y[test_index]
    nb_classes = {-1:0,0:0,1:0}
    for c in y_train:
        if(np.array_equal([0., 0., 1.],c)):
            nb_classes[0]+=1
        if(np.array_equal([0., 1., 0.],c)):
            nb_classes[-1]+=1
        if(np.array_equal([1., 0., 0.],c)):
            nb_classes[1]+=1
    print(nb_classes)
    print(len(X_train),len(X_val))
    y_train = keras.utils.to_categorical(y_train, num_classes=params['n_classes'])
    y_val = keras.utils.to_categorical(y_val, num_classes=params['n_classes'])

Fold 0:
  Train: index=[ 334  335  336 ... 5007 5008 5009]
  Test:  index=[   0    1    2 ... 3671 3672 3673]
{-1: 0, 0: 0, 1: 0}
4008 1002
Fold 1:
  Train: index=[   0    1    2 ... 5007 5008 5009]
  Test:  index=[ 334  335  336 ... 4005 4006 4007]
{-1: 0, 0: 0, 1: 0}
4008 1002
Fold 2:
  Train: index=[   0    1    2 ... 5007 5008 5009]
  Test:  index=[ 668  669  670 ... 4339 4340 4341]
{-1: 0, 0: 0, 1: 0}
4008 1002
Fold 3:
  Train: index=[   0    1    2 ... 5007 5008 5009]
  Test:  index=[1002 1003 1004 ... 4673 4674 4675]
{-1: 0, 0: 0, 1: 0}
4008 1002
Fold 4:
  Train: index=[   0    1    2 ... 4673 4674 4675]
  Test:  index=[1336 1337 1338 ... 5007 5008 5009]
{-1: 0, 0: 0, 1: 0}
4008 1002


In [10]:
# Train the model for feature extraction
# Effectuer la validation croisée
for i, (train_index, val_index) in enumerate(skf.split(X, y)):
    if(i==0):
        # Create the VGG16 model for feature extraction
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, n_channels))

        # Freeze the layers of the convolutional base
        for layer in base_model.layers:
            layer.trainable = False

        # Create the top layers for feature extraction
        model = Sequential()
        model.add(base_model)
        model.add(Flatten())
        model.add(Dense(256, activation='relu'))
        model.add(Dense(num_classes, activation='softmax'))

        # Compile the model for feature extraction
        model.compile(optimizer=Adam(learning_rate=2e-5), loss='binary_crossentropy', metrics=['accuracy'])

        # Diviser les données d'entraînement et de validation pour ce pli
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        y_train = keras.utils.to_categorical(y_train, num_classes=params['n_classes'])
        y_val = keras.utils.to_categorical(y_val, num_classes=params['n_classes'])


        # Entraîner le classifieur sur les données d'entraînement
        print(f'Fold {i}')
        scores[i]['history'] = model.fit(x=X_train,y=y_train,validation_data=(X_val,y_val),epochs=50)

        # Unfreeze the upper layers of the convolutional base
        for layer in model.layers[0].layers[15:]:
            layer.trainable = True
        
        # Compile the model for fine-tuning
        model.compile(optimizer=Adam(learning_rate=5e-6), loss='binary_crossentropy', metrics=['accuracy'])

        # Entraîner le classifieur sur les données d'entraînement
        print(f'Fold {i} fine tuning')
        scores[i]['history_fine_tuning'] = model.fit(x=X_train,y=y_train,validation_data=(X_val,y_val),epochs=100)



Fold 0
Epoch 1/50
126/126 [==============================] - 36s 114ms/step - loss: 0.6009 - accuracy: 0.6385 - val_loss: 0.5365 - val_accuracy: 0.8024
Epoch 2/50
126/126 [==============================] - 11s 85ms/step - loss: 0.5037 - accuracy: 0.7642 - val_loss: 0.4607 - val_accuracy: 0.8623
Epoch 3/50
126/126 [==============================] - 10s 83ms/step - loss: 0.4438 - accuracy: 0.7762 - val_loss: 0.4079 - val_accuracy: 0.8613
Epoch 4/50
126/126 [==============================] - 11s 83ms/step - loss: 0.4020 - accuracy: 0.7794 - val_loss: 0.3702 - val_accuracy: 0.8553
Epoch 5/50
126/126 [==============================] - 10s 83ms/step - loss: 0.3727 - accuracy: 0.7787 - val_loss: 0.3436 - val_accuracy: 0.8653
Epoch 6/50
126/126 [==============================] - 10s 82ms/step - loss: 0.3513 - accuracy: 0.7817 - val_loss: 0.3228 - val_accuracy: 0.8603
Epoch 7/50
126/126 [==============================] - 10s 83ms/step - loss: 0.3349 - accuracy: 0.7857 - val_loss: 0.3089 - val_a

In [ ]:
plt.plot(np.concatenate((history.history['accuracy'],history_fine_tuning.history['accuracy']),axis=0))
plt.plot(np.concatenate((history.history['val_accuracy'],history_fine_tuning.history['val_accuracy']),axis=0))
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(np.concatenate((history.history['loss'],history_fine_tuning.history['loss']),axis=0))
plt.plot(np.concatenate((history.history['val_loss'],history_fine_tuning.history['val_loss']),axis=0))
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [9]:
scores

{0: {'history': None, 'history_fine_tuning': None},
 1: {'history': None, 'history_fine_tuning': None},
 2: {'history': None, 'history_fine_tuning': None},
 3: {'history': <keras.callbacks.History at 0x15008932bf0>,
  'history_fine_tuning': <keras.callbacks.History at 0x15044bd61d0>},
 4: {'history': None, 'history_fine_tuning': None}}

In [12]:
np.save('history_fine_tuning_0.npy',scores[0]['history_fine_tuning'].history)

In [14]:
history=np.load('history_0.npy',allow_pickle='TRUE').item()

In [15]:
history

{'loss': [0.6076133847236633,
  0.5119550228118896,
  0.45512035489082336,
  0.41340169310569763,
  0.3831617534160614,
  0.36029306054115295,
  0.34224456548690796,
  0.32872122526168823,
  0.31772056221961975,
  0.3082601726055145,
  0.30121830105781555,
  0.2946246862411499,
  0.2893848419189453,
  0.28565722703933716,
  0.2804762125015259,
  0.2774817943572998,
  0.27415260672569275,
  0.27118176221847534,
  0.26922252774238586,
  0.2672516703605652,
  0.26537400484085083,
  0.26316073536872864,
  0.2615756094455719,
  0.2598535418510437,
  0.25900721549987793,
  0.25737398862838745,
  0.2561050355434418,
  0.25433868169784546,
  0.2543838620185852,
  0.25266116857528687,
  0.2518019378185272,
  0.25076478719711304,
  0.24994918704032898,
  0.24908329546451569,
  0.24815984070301056,
  0.24749702215194702,
  0.24651531875133514,
  0.24589426815509796,
  0.2457185834646225,
  0.2443080097436905,
  0.24430184066295624,
  0.24321138858795166,
  0.24224723875522614,
  0.243080392479896